In [1]:
#Given a mutual fund’s historical risk, returns, cost, and consistency metrics,how likely is it to perform well in the future relative to peers?
#ANS: We built a goal-based mutual fund recommendation system with return projection and risk alignment

"""
Suitability Score =
  w1 * Composite Score
+ w2 * Sharpe Ratio
+ w3 * Consistency Score
- w4 * Expense Ratio
- w5 * Risk Penalty
"""

'\nSuitability Score =\n  w1 * Composite Score\n+ w2 * Sharpe Ratio\n+ w3 * Consistency Score\n- w4 * Expense Ratio\n- w5 * Risk Penalty\n'

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel("top_30_mutual_funds_excel.xlsx")

In [4]:
df.head()

,scheme_name,min_sip,min_lumpsum,expense_ratio,fund_size_cr,fund_age_yr,fund_manager,sortino_ratio,alpha,standard_deviation,...,category,sub_category,returns_1yr,returns_3yr,returns_5yr,risk-adjusted return score,cost efficiency score,consistency score,fund stability,composite_score
0,Quant Active Fund,1000,5000,0.58,3531,10,Sanjeev Sharma,3.33,13.82,18.235557,...,Equity,Multi Cap Funds,1.0,45.5,19.9,0.070188,1.724138,23.25,2047.98,7.684121
1,Kotak Multi Asset Allocator FoF – Dynamic – Di...,1000,5000,0.13,783,10,Devender Singhal,4.33,8.04,5.786382,...,Other,FoFs Domestic,11.2,25.0,15.3,0.087065,7.692308,18.10,101.79,7.233346
2,Tata Digital India Fund,150,5000,0.31,6765,7,Meeta Shetty,2.29,6.11,23.002947,...,Equity,Sectoral / Thematic Mutual Funds,-16.0,39.0,22.1,0.150573,3.225806,11.50,2097.15,4.378371
3,Kotak India Growth Fund,0,5000,0.34,90,5,Devender Singhal,3.71,7.67,12.715432,...,Equity,Multi Cap Funds,4.8,35.5,15.6,0.123859,2.941176,20.15,30.60,6.690676
4,ICICI Pru Thematic Advantage Fund,1000,5000,0.25,958,10,Dharmesh Kakkad,4.13,6.63,12.548926,...,Other,FoFs Domestic,5.4,35.4,14.6,0.147813,4.000000,20.40,239.50,6.864500


In [5]:
df.shape

(30, 26)

In [6]:
df.describe()

,min_sip,min_lumpsum,expense_ratio,fund_size_cr,fund_age_yr,sortino_ratio,alpha,standard_deviation,beta,sharpe,risk_level,rating,returns_1yr,returns_3yr,returns_5yr,risk-adjusted return score,cost efficiency score,consistency score,fund stability,composite_score
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,626.666667,3923.333333,0.551333,6264.966667,8.733333,4.068667,11.373333,14.749867,0.869333,1.945333,5.766667,3.433333,7.263333,41.146667,16.315739,0.094426,2.713070,24.205000,3989.240333,7.986061
std,407.416870,1991.305815,0.246475,7518.172272,2.132399,0.925965,5.019094,5.611092,0.309013,0.199909,0.817200,2.028815,7.340369,10.866708,3.822083,0.074259,2.552436,5.929476,5681.524804,1.470683
min,0.000000,0.000000,0.080000,89.000000,4.000000,2.290000,5.290000,1.452201,0.530000,1.370000,2.000000,0.000000,-16.000000,14.500000,9.490726,0.019457,1.000000,11.500000,7.120000,4.378371
25%,150.000000,5000.000000,0.370000,971.750000,9.000000,3.597500,7.762500,12.702578,0.750000,1.880000,6.000000,3.000000,3.750000,35.425000,13.925000,0.059891,1.357143,20.600000,576.840000,7.025124
50%,750.000000,5000.000000,0.580000,3300.500000,10.000000,4.020000,9.940000,13.880156,0.840000,1.945000,6.000000,4.000000,7.500000,42.200000,15.600000,0.079511,1.724138,24.000000,2072.565000,7.785995
75%,1000.000000,5000.000000,0.737500,8547.750000,10.000000,4.405000,14.067500,17.215095,0.917500,2.075000,6.000000,5.000000,11.050000,45.000000,19.325000,0.095386,2.749361,27.975000,4748.167500,8.862298
max,1000.000000,5000.000000,1.000000,29953.000000,10.000000,7.270000,27.240000,27.880858,2.360000,2.300000,6.000000,5.000000,23.200000,71.400000,23.200000,0.446125,12.500000,38.400000,22464.750000,11.390875


In [7]:
df.columns.tolist()

['scheme_name',
 'min_sip',
 'min_lumpsum',
 'expense_ratio',
 'fund_size_cr',
 'fund_age_yr',
 'fund_manager',
 'sortino_ratio',
 'alpha',
 'standard_deviation',
 'beta',
 'sharpe',
 'risk_level',
 'risk_bucket',
 'amc_name',
 'rating',
 'category',
 'sub_category',
 'returns_1yr',
 'returns_3yr',
 'returns_5yr',
 'risk-adjusted return score',
 'cost efficiency score',
 'consistency score',
 'fund stability',
 'composite_score']

In [8]:
df = df.rename(columns={
    "risk-adjusted return score": "risk_adjusted_return_score",
    "cost efficiency score": "cost_efficiency_score",
    "consistency score": "consistency_score",
    "fund stability": "fund_stability"
})

In [9]:
df.columns.tolist()

['scheme_name',
 'min_sip',
 'min_lumpsum',
 'expense_ratio',
 'fund_size_cr',
 'fund_age_yr',
 'fund_manager',
 'sortino_ratio',
 'alpha',
 'standard_deviation',
 'beta',
 'sharpe',
 'risk_level',
 'risk_bucket',
 'amc_name',
 'rating',
 'category',
 'sub_category',
 'returns_1yr',
 'returns_3yr',
 'returns_5yr',
 'risk_adjusted_return_score',
 'cost_efficiency_score',
 'consistency_score',
 'fund_stability',
 'composite_score']

In [10]:
df["target_score_raw"] = (
    0.5 * df["returns_5yr"]
    + 0.3 * df["sharpe"]
    - 0.2 * df["standard_deviation"]
)

In [11]:
num_features = [
    "standard_deviation",
    "sharpe",
    "sortino_ratio",
    "expense_ratio",
    "fund_size_cr",
    "consistency_score",
    "cost_efficiency_score",
    "fund_stability",
    "alpha",
    "beta"
]

cat_features = [
    "category",
    "risk_bucket"
]

In [12]:
from sklearn.model_selection import train_test_split

X = df[num_features + cat_features]
y = df["target_score_raw"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


In [14]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [15]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [16]:
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

In [17]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

In [18]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", model)
])

In [19]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['standard_deviation',
                                                   'sharpe', 'sortino_ratio',
                                                   'expense_ratio',
                                                   'fund_size_cr',
                                                   'consistency_score',
                                                   'cost_efficiency_score',
                                                   'fund_stability', 'alpha',
                                                   'beta']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['category',
                                                   'risk_bucket'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [23]:
df["predicted_raw_score"] = pipeline.predict(X)
df.sort_values("predicted_raw_score", ascending=False)[
    ["scheme_name", "predicted_raw_score", "target_score_raw"]
].head(5)

,scheme_name,predicted_raw_score,target_score_raw
19,Quant Multi Asset Fund,7.593713,8.111664
26,AXIS Small Cap Fund,7.074105,7.778341
22,Quant Mid Cap Fund,7.008190,7.445424
18,Quant Tax Plan- Direct Growth,6.966759,7.267289
29,Parag Parikh Flexi Cap Fund,6.842504,6.802153


In [24]:
df[["returns_5yr", "sharpe", "standard_deviation", "target_score_raw"]].corr()

,returns_5yr,sharpe,standard_deviation,target_score_raw
returns_5yr,1.000000,-0.004150,0.488414,0.813183
sharpe,-0.004150,1.000000,0.082279,-0.024046
standard_deviation,0.488414,0.082279,1.000000,-0.109745
target_score_raw,0.813183,-0.024046,-0.109745,1.000000


with Z-score target

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[["z_returns_5yr", "z_sharpe", "z_std_dev"]] = scaler.fit_transform(
    df[["returns_5yr", "sharpe", "standard_deviation"]]
)


In [26]:
df["target_score_z"] = (
    0.5 * df["z_returns_5yr"]
    + 0.3 * df["z_sharpe"]
    - 0.2 * df["z_std_dev"]
)


In [27]:
y = df["target_score_z"]
X = df[num_features + cat_features]


In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)


In [29]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['standard_deviation',
                                                   'sharpe', 'sortino_ratio',
                                                   'expense_ratio',
                                                   'fund_size_cr',
                                                   'consistency_score',
                                                   'cost_efficiency_score',
                                                   'fund_stability', 'alpha',
                                                   'beta']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['category',
                                                   'risk_bucket'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [30]:
y_pred = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

mae, rmse, r2


(0.467453907625176, np.float64(0.5998512234061355), 0.10951325878790352)

In [31]:
from scipy.stats import spearmanr

spearman_corr, _ = spearmanr(y_test, y_pred)
spearman_corr


np.float64(0.11904761904761905)

In [32]:
df["predicted_z_score"] = pipeline.predict(X)

top_actual = (
    df.sort_values("target_score_z", ascending=False)
      .head(5)["scheme_name"]
)

top_predicted = (
    df.sort_values("predicted_z_score", ascending=False)
      .head(5)["scheme_name"]
)

top_actual, top_predicted, set(top_actual).intersection(set(top_predicted))


(17        Quant Infrastructure Fund
 21              Quant Absolute Fund
 16             Quant Small Cap Fund
 26              AXIS Small Cap Fund
 18    Quant Tax Plan- Direct Growth
 Name: scheme_name, dtype: object,
 21              Quant Absolute Fund
 18    Quant Tax Plan- Direct Growth
 26              AXIS Small Cap Fund
 16             Quant Small Cap Fund
 17        Quant Infrastructure Fund
 Name: scheme_name, dtype: object,
 {'AXIS Small Cap Fund',
  'Quant Absolute Fund',
  'Quant Infrastructure Fund',
  'Quant Small Cap Fund',
  'Quant Tax Plan- Direct Growth'})

In [33]:
df[[
    "z_returns_5yr",
    "z_sharpe",
    "z_std_dev",
    "target_score_z"
]].corr()


,z_returns_5yr,z_sharpe,z_std_dev,target_score_z
z_returns_5yr,1.000000,-0.004150,0.488414,0.770156
z_sharpe,-0.004150,1.000000,0.082279,0.540489
z_std_dev,0.488414,0.082279,1.000000,0.132287
target_score_z,0.770156,0.540489,0.132287,1.000000


In [34]:
df["final_score"] = (
    0.4 * df["predicted_z_score"]
    + 0.6 * df["composite_score"]
)


In [54]:
risk_map = {
    "Low": ["Low Risk", "Moderately Low"],
    "Moderate": ["Moderate", "Moderately Low"],
    "High": ["High Risk", "Moderate"]
}



In [55]:
def filter_by_risk(df, user_risk):
    allowed = risk_map[user_risk]
    return df[df["risk_bucket"].isin(allowed)]



In [56]:
df_risk = filter_by_risk(df, "Moderate")
len(df_risk)


2

In [57]:
def create_return_band(r):
    if r < 10:
        return "Low"
    elif r < 15:
        return "Moderate"
    else:
        return "High"


In [58]:
df["return_band"] = df["returns_5yr"].apply(create_return_band)


In [59]:
df["return_band"].value_counts()


return_band
High        18
Moderate     9
Low          3
Name: count, dtype: int64

In [46]:
return_bands = {
    "Conservative": (6, 9),
    "Moderate": (9, 13),
    "Aggressive": (13, 18)
}

In [47]:
def filter_by_return(df, band):
    low, high = return_bands[band]
    return df[(df["returns_5yr"] >= low) & (df["returns_5yr"] <= high)]

In [48]:
def recommend_top_funds(
    df,
    user_risk,
    return_band,
    top_n=5
):
    df_filtered = filter_by_risk(df, user_risk)
    df_filtered = filter_by_return(df_filtered, return_band)

    return (
        df_filtered
        .sort_values("final_score", ascending=False)
        .head(top_n)
        [["scheme_name", "final_score", "returns_5yr", "risk_bucket"]]
    )


In [49]:
def sip_future_value(monthly_sip, annual_rate, years):
    r = annual_rate / 100 / 12
    n = years * 12
    fv = monthly_sip * (((1 + r)**n - 1) / r) * (1 + r)
    return round(fv, 2)


In [50]:
def lumpsum_future_value(amount, annual_rate, years):
    r = annual_rate / 100
    fv = amount * ((1 + r) ** years)
    return round(fv, 2)


In [51]:
def attach_projection(
    df_reco,
    investment_type,
    amount,
    years
):
    results = []

    for _, row in df_reco.iterrows():
        rate = row["returns_5yr"]

        if investment_type == "SIP":
            final_value = sip_future_value(amount, rate, years)
        else:
            final_value = lumpsum_future_value(amount, rate, years)

        results.append({
            "scheme_name": row["scheme_name"],
            "expected_cagr": rate,
            "investment_years": years,
            "final_value": final_value
        })

    return pd.DataFrame(results)


In [52]:
top_funds = recommend_top_funds(
    df,
    user_risk="Moderate",
    return_band="Moderate",
    top_n=5
)

projection = attach_projection(
    top_funds,
    investment_type="SIP",
    amount=5000,
    years=10
)

projection


""


In [60]:
df_risk = filter_by_risk(df, "Moderate")
print(len(df_risk))


2


In [61]:
df_return = df[df["return_band"] == "Moderate"]
print(len(df_return))


9


In [62]:
df_both = df_risk[df_risk["return_band"] == "Moderate"]
print(len(df_both))


1


In [71]:
top_funds = recommend_top_funds(
    df,
    user_risk="Moderate",
    return_band="Moderate",
    top_n=5
)

top_funds


,scheme_name,final_score,returns_5yr,risk_bucket
14,ICICI Pru Asset Allocator Fund,4.000079,12.8,Moderately Low


In [68]:
projection = attach_projection(
    top_funds,
    investment_type="SIP",
    amount=10000,
    years=10
)
